In [1]:
import pandas as pd
import numpy as np
import os

from thefuzz import fuzz #new fuzzywuzzy project https://github.com/seatgeek/thefuzz
import textdistance

# "Library"

In [2]:
def isNaN(value):
    if type(value)==str:
        return (value.upper() == 'NAN')
    else:
        return np.isnan(value)
    
def areEquivalentValues(val1, val2, equivalences):
    return (val1==val2 or (val1,val2) in equivalences) or ((val2,val1) in equivalences)

In [3]:
def compatibleRanges(row1, row2, options, verbose=False):
    compatible = True #need to be compatible on all available fields (can be changed for 'or')
    allowNa = options['na.action'] == 'all'
    
    #for debugging
    val1 = val2 = 'NAN'
    
    for field in options['fields'].split(' '):        
        if field not in row1 or field not in row2:
            continue
        
        if isNaN(row1[field]) or isNaN(row2[field]):
            compatible &= allowNa
        else:
            compatible &= abs(int(row1[field]) - int(row2[field])) <= int(options['parameter'])
            val1 = int(row1[field])
            val2 = int(row2[field])
            
    #for debugging
    if verbose: 
        print(f"\tval1:{val1}, val2:{val2}, allowNa: {allowNa}, range:{int(options['parameter'])}")
    
    return compatible

def compatibleCategory(row1, row2, options, verbose=False):
    compatible = True #need to be compatible on all available fields (can be changed for 'or')
    allowNa = options['na.action'].upper() == 'ALL'
    equivalences = None
    
    #for debugging
    val1 = val2 = 'NAN'
    
    if not isNaN(options['parameter']):
        equivalencesDF = pd.read_csv(f"{configPath}/{options['parameter']}")
        equivalences = list(equivalencesDF.itertuples(index=False, name=None))
        # WARNING, there may be a better way to do this other than creating tuples but we should have a scheme
        # to follow, in order to grab the columns without having to specify their names
    
    for field in options['fields'].split(' '):
        if field not in row1 or field not in row2:
            continue
        
        if isNaN(row1[field]) or isNaN(row2[field]):
            compatible &= allowNa
        elif equivalences:
            compatible &= areEquivalentValues(row1[field], row2[field], equivalences)
            val1 = row1[field]
            val2 = row2[field]
    
    #for debugging
    if verbose: 
        print(f"\tval1:{val1}, val2:{val2}, allowNa: {allowNa}")
    
    return compatible

In [4]:
def areCompatibles(row1, row2, verbose=False):
    compatible = True
        
    for variable, options in comparisonSettings.items():    
        if options['consider'].lower() == 'yes':
            if verbose:
                print(f"{variable} compatible?")
            varCompatible = True

            if options['type'] == 'range':
                varCompatible = compatibleRanges(row1, row2, options, verbose)
            elif options['type'] == 'categorical':
                varCompatible = compatibleCategory(row1, row2, options, verbose)

            compatible &= varCompatible
            if verbose:
                print(f"\tresult: {varCompatible}")
    if verbose:
        print(f"Candidates? {compatible} \n\n")
    return compatible

In [5]:
def areCandidates(row1, row2, verbose=False):
    candidates = True

    for scheme, key1, key2, threshold in schemesConfig.itertuples(index=False):
        #check for valid scheme and keys
        if scheme not in schemes:
            if verbose:
                print(f"scheme not used: {scheme}")
            continue
        if verbose:
            print(f"scheme {scheme}")
        if key1 not in base1.columns:
            print(f"invalid key on database 1 {key1}, skipping")
            continue
        if key2 not in base2.columns:
            print(f"invalid key on database 2 {key2}, skipping")
            continue

        value1 = row1[key1]
        value2 = row2[key2]
        
        information = {}
        
        for algorithmName, algorithm in comparisonAlgorithms.items():
            result = algorithm(value1, value2)
            candidates &= (result >= threshold)

            information[f"{algorithmName}_{scheme}"] = result
    return candidates, pd.Series(information)

# Config

In [6]:
#config files
configPath = os.path.join(os.getcwd(), 'config')
dictionariesPath = os.path.join(os.getcwd(), 'dicts')

In [7]:
#input
inputPath = os.path.join(os.getcwd(), 'example/instances')

base1 = pd.read_csv(f"{inputPath}/inst1/base_inst1.csv")
base2 = pd.read_csv(f"{inputPath}/inst2/base_inst2.csv")

#output
outputPath = os.path.join(inputPath, 'matches')
if not os.path.exists(outputPath):
    os.makedirs(outputPath)
    
outputFileName = 'candidateList'

In [8]:
variableFields = pd.read_csv(f"{configPath}/variableFields.csv").set_index('variable')
compatibleData = pd.read_csv(f"{configPath}/compatible_data.csv").set_index('variable')

comparisonSettings = compatibleData.join(variableFields).to_dict(orient='index')

compatibleData.join(variableFields) #just to visualize

,type,parameter,consider,na.action,fields
variable,,,,,
Case.ID,Case.ID,NaN,no,NaN,Case.ID
cod,cod,NaN,no,NaN,cod
nationality,categorical,natEquivalences.csv,yes,all,Nat_PROC Nat_2_PROC
sex,categorical,NaN,yes,all,Sex_PROC
age,range,6,yes,all,Age_PROC Age_2_PROC


In [9]:
schemes = pd.read_csv(f"{configPath}/select_schemes.csv", names=['schemes'])['schemes'].to_list() 
print(f"schemes {schemes}")
#TODO add option for "fast scheme"

schemesConfig = pd.read_csv(f"{configPath}/info_scheme.csv")
schemesConfig.rename(columns = {'listA.column': 'key1', 'listB.column': 'key2'}, inplace=True)
if schemesConfig.dtypes['threshold'] == str:
    schemesConfig.threshold = schemesConfig.threshold.str.replace(',', '.').astype(float)
schemesConfig

schemes ['A_A', 'F_F']


,scheme,key1,key2,threshold
0,A_A,Name_A,Name_A,0.3
1,F_F,Father_name_A,Father_name_A,0.0
2,M_M,Mother_name_A,Mother_name_A,0.0
3,AD_AD,Adress_A,Adress_A,0.0
4,Ph_Ph,Phone_1_PROC Phone_2_PROC Phone_3_PROC,Phone_1_PROC Phone_2_PROC Phone_3_PROC,0.0


In [10]:
comparisonAlgorithms = {
    'Levenshtein': textdistance.levenshtein.normalized_similarity,
    'DL': textdistance.damerau_levenshtein.normalized_similarity,
    'fuzzy_ratio' : fuzz.ratio
} #TODO add the rest/more

# Script

## Find compatible rows

In [11]:
compatiblesDF = base1.apply(lambda row1: base2.apply(lambda row2: areCompatibles(row1, row2), axis=1), axis=1)
i1, i2 = (compatiblesDF.values).nonzero()
compatibles = list(zip(i1, i2))

compatibles

[(2, 1), (2, 7), (4, 12), (4, 23), (5, 24)]

## Use the schemes configured on the compatible pairs

In [12]:
candidatesList = []

for i1, i2 in compatibles:
    row1 = base1.iloc[i1]
    row2 = base2.iloc[i2]
    
    candidates, information = areCandidates(row1, row2)

    if candidates:
        #rename columns as base1_ and base2_
        row1.set_axis(['base1_'+x for x in base1.columns], inplace=True)
        row2.set_axis(['base2_'+x for x in base2.columns], inplace=True)
        
        candidatesList.append(pd.concat([row1, row2, information]))

candidatesDF = pd.concat(candidatesList, axis=1).T

candidatesDF.to_csv(f"{outputPath}/candidatesList.csv", index=False)
candidatesDF

invalid key on database 1 Father_name_A, skipping
invalid key on database 1 Father_name_A, skipping
invalid key on database 1 Father_name_A, skipping
invalid key on database 1 Father_name_A, skipping
invalid key on database 1 Father_name_A, skipping


,base1_cod,base1_orden.base,base1_Case.ID,base1_Name_1,base1_Nat,base1_Sex,base1_Age,base1_Rol,base1_Name_1_PROC,base1_Nat_PROC,...,base2_Name_1_PROC,base2_Nat_PROC,base2_Sex_PROC,base2_Age_PROC,base2_Rol_PROC,base2_Name_A,base2_Name_B,Levenshtein_A_A,DL_A_A,fuzzy_ratio_A_A
0,BA1,NaN,BA1-06,Juan Perez,Mali,NaN,15,Missing,juan perez,MALI,...,joan perez,MAURITANIA,M,18.0,NaN,joan perez,joan perez,0.9,0.9,90.0
1,BA1,NaN,BA1-06,Juan Perez,Mali,NaN,15,Missing,juan perez,MALI,...,jose perez,MALI,M,15.0,NaN,jose perez,jose perez,0.7,0.7,70.0
2,BA1,NaN,BA1-11,Juana Morales,Yemen,F,44,Missing,juana morales,YEMEN,...,juan morale,YEMEN,NaN,40.0,NaN,juan morale,juan morale,0.846154,0.846154,92.0
3,BA1,NaN,BA1-11,Juana Morales,Yemen,F,44,Missing,juana morales,YEMEN,...,juana morales,YEMEN,NaN,NaN,NaN,juana morales,juana morales,1.0,1.0,100.0
4,BA1,NaN,BA1-12,Esteban Quito,Seychelles,M,9,Missing,esteban quito,SEYCHELLES,...,esteban quito,SEYCHELLES,NaN,NaN,NaN,esteban quito,esteban quito,1.0,1.0,100.0


# Only for checking

In [13]:
exBase1 = base1.iloc[4]
exBase2 = base2.iloc[12]

In [14]:
exBase1

cod                      BA1
orden.base               NaN
Case.ID               BA1-11
Name_1         Juana Morales
Nat                    Yemen
Sex                        F
Age                       44
Rol                  Missing
Name_1_PROC    juana morales
Nat_PROC               YEMEN
Sex_PROC                   F
Age_PROC                  44
Rol_PROC             missing
Name_A         juana morales
Name_B         juana morales
Name: 4, dtype: object

In [15]:
exBase2

cod                    BA2
orden.base             NaN
Case.ID             BA2-13
Name_1         Juan Morale
Nat                  Yemen
Sex                    NaN
Age                     40
Rol                    NaN
Name_1_PROC    juan morale
Nat_PROC             YEMEN
Sex_PROC               NaN
Age_PROC              40.0
Rol_PROC               NaN
Name_A         juan morale
Name_B         juan morale
Name: 12, dtype: object

In [16]:
#TEST

#Stringdist (R) Functions
    #Levenshtein distance
print("levenshtein normalized similarity", textdistance.levenshtein.normalized_similarity('test', 'tast'))
    
    #D-L 
        #The restricted Damerau-Levenshtein distance is not a true distance metric because it does not 
        #satisfy the triangle inequality. This makes it a poor choice for applications that involve evaluating 
        #the similarity of more than two strings, such as clustering.
print("damerau-levenshtein normalized", textdistance.damerau_levenshtein.normalized_similarity('test', 'tast'))

    #D-L-FULL ???
    
    #LongestCommonSubstring 
    #falta encontrar uno que haga tal cuál se quiere, sino se puede calcular, pero con cuidado
ex1 = 'Jonh Smit'
ex2 = 'John Smith'
substr = textdistance.lcsstr(ex1, ex2)
print(substr)
print("longest common substring", 1-len(substr)/(len(ex1)+len(ex2))) #TODO
    
    #GramX

#FuzzyWuzzyFunctions
    #Ratio
print("fuzz.ratio('test', 'tast')", fuzz.ratio('test', 'tast'))
    #partial_ratio
print("fuzz.partial_ratio('test', 'tast!')", fuzz.partial_ratio('test', 'tast!'))
print("fuzz.partial_ratio('test', 'test!')", fuzz.partial_ratio('test', 'test!'))
    #token_sort_ratio
print("fuzz.ratio('fuzzy wuzzy was a bear', 'wuzzy fuzzy was a bear')", fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear"))
print("fuzz.token_sort_ratio('fuzzy wuzzy was a bear', 'wuzzy fuzzy was a bear')", fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear"))
    #WRATIO == Set Ratio?
print("fuzz.token_sort_ratio('fuzzy was a bear', 'fuzzy fuzzy was a bear')", fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear"))
print("fuzz.token_set_ratio('fuzzy was a bear', 'fuzzy fuzzy was a bear')", fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear"))

levenshtein normalized similarity 0.75
damerau-levenshtein normalized 0.75
 Smit
longest common substring 0.736842105263158
fuzz.ratio('test', 'tast') 75
fuzz.partial_ratio('test', 'tast!') 75
fuzz.partial_ratio('test', 'test!') 100
fuzz.ratio('fuzzy wuzzy was a bear', 'wuzzy fuzzy was a bear') 91
fuzz.token_sort_ratio('fuzzy wuzzy was a bear', 'wuzzy fuzzy was a bear') 100
fuzz.token_sort_ratio('fuzzy was a bear', 'fuzzy fuzzy was a bear') 84
fuzz.token_set_ratio('fuzzy was a bear', 'fuzzy fuzzy was a bear') 100


In [17]:
print(base1.shape)
base1

(6, 15)


,cod,orden.base,Case.ID,Name_1,Nat,Sex,Age,Rol,Name_1_PROC,Nat_PROC,Sex_PROC,Age_PROC,Rol_PROC,Name_A,Name_B
0,BA1,NaN,BA1-01,Jorge Perez,Denmark,M,11,Missing,jorge perez,DENMARK,M,11,missing,jorge perez,jorge perez
1,BA1,NaN,BA1-05,Oscar Alvarez,Jordan,M,44,Missing,oscar alvarez,JORDAN,M,44,missing,oscar alvarez,oscar alvarez
2,BA1,NaN,BA1-06,Juan Perez,Mali,NaN,15,Missing,juan perez,MALI,NaN,15,missing,juan perez,juan perez
3,BA1,NaN,BA1-07,Maria Urcupiña,Peru,F,18,Missing,maria urcupiña,PERU,F,18,missing,maria urcupiña,maria urcupiña
4,BA1,NaN,BA1-11,Juana Morales,Yemen,F,44,Missing,juana morales,YEMEN,F,44,missing,juana morales,juana morales
5,BA1,NaN,BA1-12,Esteban Quito,Seychelles,M,9,Missing,esteban quito,SEYCHELLES,M,9,missing,esteban quito,esteban quito


In [18]:
print(base2.shape)
base2.head(14)

(25, 15)


,cod,orden.base,Case.ID,Name_1,Nat,Sex,Age,Rol,Name_1_PROC,Nat_PROC,Sex_PROC,Age_PROC,Rol_PROC,Name_A,Name_B
0,BA2,NaN,BA2-01,Aarón Ramirez,Morocco,M,20,NaN,aaron ramirez,MOROCCO,M,20.0,NaN,aaron ramirez,aaron ramirez
1,BA2,NaN,BA2-02,Joan Perez,Mauritania,M,18,NaN,joan perez,MAURITANIA,M,18.0,NaN,joan perez,joan perez
2,BA2,NaN,BA2-03,Marina Ruana,Nepal,F,44,NaN,marina ruana,NEPAL,F,44.0,NaN,marina ruana,marina ruana
3,BA2,NaN,BA2-04,Carl Sorlovsky,Netherlands,M,50,NaN,carl sorlovsky,NETHERLANDS,M,50.0,NaN,carl sorlovsky,carl sorlovsky
4,BA2,NaN,BA2-05,Mariano Hernandez,holand,M,78,NaN,mariano hernandez,NETHERLANDS,M,78.0,NaN,mariano hernandez,mariano hernandez
5,BA2,NaN,BA2-06,Carla,Norway,F,10,NaN,carla,NORWAY,F,10.0,NaN,carla,carla
6,BA2,NaN,BA2-07,John Buffalo,romania (stateless),NaN,30,NaN,john buffalo,ROMANIA,NaN,30.0,NaN,john buffalo,john buffalo
7,BA2,NaN,BA2-02,Jose Perez,Mali,M,15,NaN,jose perez,MALI,M,15.0,NaN,jose perez,jose perez
8,BA2,NaN,BA2-09,Alina Alvez,siera leone,F,NaN,NaN,alina alvez,SIERRA LEONE,F,NaN,NaN,alina alvez,alina alvez
9,BA2,NaN,BA2-10,Inés Martinez,slovenia,F,42,NaN,ines martinez,SLOVENIA,F,42.0,NaN,ines martinez,ines martinez
